In [42]:
file = './data/test.txt'

In [43]:
user2id = {}

In [44]:
def read_users(file):        
        with open(file, 'r') as f:            
            lines = f.readlines()
            prev_user = int(lines[0].split('\t')[0])
            visit_cnt = 0
            for line in lines:
                tokens = line.strip().split('\t')
                print(tokens)
                user = int(tokens[0])
                if user == prev_user:
                    visit_cnt += 1
                else:
                    if visit_cnt >= 3:
                        user2id[prev_user] = len(user2id)
                    #else:
                    #    print('discard user {}: to few checkins ({})'.format(prev_user, visit_cnt))
                    prev_user = user
                    visit_cnt = 1
                    if len(user2id) >= 4:
                        break # restrict to max users

In [45]:
read_users(file)

['75', '2013-12-20T15:24:43Z', '7.05008100', '125.58815300', '1320272']
['75', '2013-12-13T03:34:24Z', '7.09084200', '125.61153900', '3519110']
['75', '2013-11-08T10:38:39Z', '7.09067200', '125.61175300', '3519671']
['65', '2013-12-20T15:24:43Z', '7.05008100', '125.58815300', '1320272']
['65', '2013-12-13T03:34:24Z', '7.09084200', '125.61153900', '3519110']
['65', '2013-11-08T10:38:39Z', '7.09067200', '125.61175300', '3519671']
['55', '2013-12-20T15:24:43Z', '7.05008100', '125.58815300', '1320272']
['55', '2013-12-13T03:34:24Z', '7.09084200', '125.61153900', '3519110']
['55', '2013-11-08T10:38:39Z', '7.09067200', '125.61175300', '3519671']


In [46]:
user2id

{75: 0, 65: 1}

In [47]:
user2id = {}

In [57]:
def read_users(file):
    with open(file, 'r') as f:
        prev_user = None
        visit_cnt = 0
        break_occurred = False

        for line in f:
            tokens = line.strip().split('\t')
            user = int(tokens[0])

            if prev_user is None:
                prev_user = user
                visit_cnt = 1
                continue

            if user == prev_user:
                visit_cnt += 1
            else:
                if visit_cnt >= 3:
                    user2id[prev_user] = len(user2id)
                # else:
                #     print(f'discard user {prev_user}: too few checkins ({visit_cnt})')
                prev_user = user
                visit_cnt = 1
                if len(user2id) >= 4:
                    break_occurred = True
                    break
        # 处理最后一个用户
        if not break_occurred and prev_user is not None and visit_cnt >= 3:
            user2id[prev_user] = len(user2id)  

In [58]:
read_users(file)

In [59]:
user2id

{75: 0, 65: 1, 55: 2}

In [60]:
user2id[75]

0

In [61]:
user2id.get(75)

0

In [62]:
from datetime import datetime

In [64]:
time =  '2013-12-20T15:24:43Z'
time = datetime.strptime(time, "%Y-%m-%dT%H:%M:%SZ")
time

datetime.datetime(2013, 12, 20, 15, 24, 43)

In [65]:
datetime(1970,1,1)

datetime.datetime(1970, 1, 1, 0, 0)

In [71]:
poi2id = {}
users = []
times = []
coords = []
locs = []

In [72]:
def read_pois(file):
    '''
    self.users   → [user0_id, user1_id, ..., userN_id]           # 用户 ID 列表
    self.times   → [[t1, t2, ...], [t1, t2, ...], ...]           # 每个用户的签到时间序列（按时间倒序）
    self.coords  → [[(lat, lon), ...], [(lat, lon), ...], ...]   # 每个用户的签到坐标序列（按时间倒序）
    self.locs    → [[loc1, loc2, ...], [loc1, loc2, ...], ...]   # 每个用户的新位置编号序列（按时间倒序）
    self.poi2id  → {原始location_id: 新ID}                       # 全部位置的映射表
    '''
    with open(file, 'r') as f:
        # store location ids
        user_time = []
        user_coord = []
        user_loc = []
        
        line = f.readline()
        prev_user = int(line.strip().split('\t')[0])
        prev_user_id = user2id.get(prev_user)
        f.seek(0)
        for line in f:
            tokens = line.strip().split('\t')
            user = int(tokens[0])
            if user2id.get(user) is None:
                continue # user is not of interest
            user_id = user2id.get(user)
            # datetime.strptime(tokens[1], "%Y-%m-%dT%H:%M:%SZ"): datetime.datetime(2013, 12, 20, 15, 24, 43)
            # T 是 ISO 8601 标准中用于连接日期和时间的分隔符
            # Z 表示时间是在 UTC 时间（零时区）     
            time = (datetime.strptime(tokens[1], "%Y-%m-%dT%H:%M:%SZ") - datetime(1970, 1, 1)).total_seconds() # unix seconds
            lat = float(tokens[2])
            long = float(tokens[3])
            coord = (lat, long)            
            location = int(tokens[4]) # location nr
            if poi2id.get(location) is None: # get-or-set locations
                poi2id[location] = len(poi2id)
            location_id = poi2id.get(location)
    
            if user_id == prev_user_id:
                # insert in front!
                user_time.insert(0, time)
                user_coord.insert(0, coord)
                user_loc.insert(0, location_id)
            else:
                users.append(prev_user_id)
                times.append(user_time)
                coords.append(user_coord)
                locs.append(user_loc)
                
                # restart:
                prev_user_id = user_id 
                user_time = [time]
                user_coord = [coord]
                user_loc = [location_id] 
                
        # process also the latest user in the for loop
        users.append(prev_user_id)
        times.append(user_time)
        coords.append(user_coord)
        locs.append(user_loc)

In [73]:
read_pois(file)

In [74]:
print(poi2id)
print(users)
print(times)
print(coords)
print(locs)

{1320272: 0, 3519110: 1, 3519671: 2}
[0, 1, 2]
[[1383907119.0, 1386905664.0, 1387553083.0], [1383907119.0, 1386905664.0, 1387553083.0], [1383907119.0, 1386905664.0, 1387553083.0]]
[[(7.090672, 125.611753), (7.090842, 125.611539), (7.050081, 125.588153)], [(7.090672, 125.611753), (7.090842, 125.611539), (7.050081, 125.588153)], [(7.090672, 125.611753), (7.090842, 125.611539), (7.050081, 125.588153)]]
[[2, 1, 0], [2, 1, 0], [2, 1, 0]]
